In [54]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D, BatchNormalization


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(tf.__version__)


2.8.2


In [55]:
submission = pd.read_csv("/content/submission.csv")
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [56]:
traindat = pd.read_csv("/content/train.csv.zip")
traindat.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
testdat = pd.read_csv("/content/test.csv.zip")
testdat.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
Y = traindat['label']
X = traindat.drop(['label'], axis = 1)
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
Y = np.array(Y)
X = np.array(X)/255.
X = np.array([x.reshape((28, 28, 1)) for x in X])

X = np.append(X, X, axis = 0)
Y = np.append(Y, Y, axis = 0)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 40)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(67200, 28, 28, 1)
(16800, 28, 28, 1)
(67200,)
(16800,)


In [60]:
model = Sequential([
  
    Conv2D(input_shape =(28,28, 1), filters = 128, kernel_size = (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
    BatchNormalization(),
    Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'sparse_categorical_crossentropy')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 128)       1280      
                                                                 
 batch_normalization_5 (Batc  (None, 26, 26, 128)      512       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 128)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 128)       147584    
                                                                 
 batch_normalization_6 (Batc  (None, 11, 11, 128)      512       
 hNormalization)                                                 
                                                      

In [61]:
history = model.fit(X_train, y_train,batch_size = 10000, epochs = 35, validation_data  = (X_test, y_test))

Epoch 1/35
7/7 [==============================] - 21s 3s/step - loss: 1.4108 - accuracy: 0.5618 - val_loss: 2.1806 - val_accuracy: 0.7392
Epoch 2/35
7/7 [==============================] - 8s 1s/step - loss: 0.4064 - accuracy: 0.8971 - val_loss: 2.1088 - val_accuracy: 0.2135
Epoch 3/35
7/7 [==============================] - 8s 1s/step - loss: 0.2269 - accuracy: 0.9428 - val_loss: 2.2980 - val_accuracy: 0.1108
Epoch 4/35
7/7 [==============================] - 8s 1s/step - loss: 0.1476 - accuracy: 0.9627 - val_loss: 2.5037 - val_accuracy: 0.1108
Epoch 5/35
7/7 [==============================] - 8s 1s/step - loss: 0.1075 - accuracy: 0.9722 - val_loss: 2.7093 - val_accuracy: 0.1108
Epoch 6/35
7/7 [==============================] - 8s 1s/step - loss: 0.0844 - accuracy: 0.9781 - val_loss: 2.9234 - val_accuracy: 0.1108
Epoch 7/35
7/7 [==============================] - 8s 1s/step - loss: 0.0683 - accuracy: 0.9820 - val_loss: 3.1677 - val_accuracy: 0.1108
Epoch 8/35
7/7 [========================

In [62]:
model.fit(X, Y, epochs = 10)

Epoch 1/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0898 - accuracy: 0.9737
Epoch 2/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0456 - accuracy: 0.9866
Epoch 3/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0320 - accuracy: 0.9906
Epoch 4/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0259 - accuracy: 0.9923
Epoch 5/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0185 - accuracy: 0.9945
Epoch 6/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0180 - accuracy: 0.9945
Epoch 7/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0126 - accuracy: 0.9961
Epoch 8/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0119 - accuracy: 0.9965
Epoch 9/10
2625/2625 [==============================] - 15s 6ms/step - loss: 0.0107 - accuracy: 0.9969
Epoch 10/10
2625/2625 [==============================] - 15s 6ms/step - l

In [63]:
testdat = np.array(testdat)/255.
testdat = np.array([x.reshape(28, 28, 1) for x in testdat])
print(testdat.shape)

(28000, 28, 28, 1)


In [64]:
test_preds = model.predict(testdat)
print(test_preds.shape)
test_preds = np.array([np.argmax(y) for y in test_preds])
print(test_preds.shape)

(28000, 10)
(28000,)


In [65]:
testId = [*range(1, len(test_preds)+1)]

In [66]:
output = pd.DataFrame({'ImageId': testId,
                      'Label': test_preds})
output.to_csv('submission.csv', index=False)